### The $\hat{A}$-genus

Gives all partitions of a nonempty set $S$. (A partition of $S$ is a set of nonempty disjoint subsets of x whose union is $S$.)

In [1]:
def set_partitions(S):
    if len(S) == 1:
        yield [S]
    else:
        for p in set_partitions(S[1:]):
            yield [[S[0]]] + p
            for i in range(0, len(p)):
                yield p[:i] + [[S[0]] + p[i]] + p[i+1:]

In [2]:
for x in set_partitions([1..3]):
    print(x)

[[1], [2], [3]]
[[1, 2], [3]]
[[2], [1, 3]]
[[1], [2, 3]]
[[1, 2, 3]]


Gives $(-1)^{n-1}$ times the coefficient of $x^{n-1}$ in $\frac{d}{dx} \log\!\left( \frac{\sqrt{x}/2}{\sinh(\sqrt{x}/2)} \right)$, needed to define the $\hat{A}$-genus. (Defined for $n \geq 1$.)

In [3]:
@cached_function
def ahatfunc(n):
    return (-1)^n * bernoulli(2 * n) / (2 * factorial(2 * n))

In [4]:
[ahatfunc(n) for n in (1..10)]

[-1/24,
 -1/1440,
 -1/60480,
 -1/2419200,
 -1/95800320,
 -691/2615348736000,
 -1/149448499200,
 -3617/21341245685760000,
 -43867/10218188434341888000,
 -174611/1605715325396582400000]

Gives the coefficient of $p^J$ in the $\hat{A}$-genus. (Defined for any integer partition $J$.)

In [5]:
@cached_function
def ahatcoeff(J):
    if len(J) == 0:
        return 1
    prefactor = (-1)^len(J) / product(factorial(m) for m in J.to_exp())
    return prefactor * sum(
        (-1)^len(pi) * product(
            factorial(len(block) - 1) * ahatfunc(sum(block))
        for block in pi)
    for pi in set_partitions(J))

In [6]:
for n in (0..5):
    print({J: ahatcoeff(J) for J in Partitions(n)})

{[]: 1}
{[1]: -1/24}
{[2]: -1/1440, [1, 1]: 7/5760}
{[3]: -1/60480, [2, 1]: 11/241920, [1, 1, 1]: -31/967680}
{[4]: -1/2419200, [3, 1]: 1/907200, [2, 2]: 13/29030400, [2, 1, 1]: -113/58060800, [1, 1, 1, 1]: 127/154828800}
{[5]: -1/95800320, [4, 1]: 53/1916006400, [3, 2]: 1/45619200, [3, 1, 1]: -61/1277337600, [2, 2, 1]: -311/7664025600, [2, 1, 1, 1]: 1073/15328051200, [1, 1, 1, 1, 1]: -73/3503554560}


Gives the $\hat{A}$-genus in degrees $0, \dots, n$, each as a polynomial in the Pontryagin classes.

In [15]:
@cached_function
def ahat(n):
    Sym = SymmetricFunctions(QQ).elementary()
    return [Sym._from_dict({J: ahatcoeff(J) for J in Partitions(m)}) for m in (0..n)]

In [14]:
pretty_print(ahat(3))

[e[],
 -1/24*e[1],
 7/5760*e[1, 1] - 1/1440*e[2],
 -31/967680*e[1, 1, 1] + 11/241920*e[2, 1] - 1/60480*e[3]]

In this output, $e_J$ represents $p^J$. For example, $e_{1,1}$ represents $p_1^2$. We will always represent polynomials in the Pontryagin classes in this form.

### The $ph(\pi^r)$ classes

Gives the coefficient of $x^{n-1}$ in $\frac{d}{dx} \log\!\left( 1 + t(e^{\sqrt{x}} + e^{-\sqrt{x}} - 2) \right)$. The result is a polynomial in $t$ of degree $n$. (Defined for $n \geq 1$.)

In [7]:
@cached_function
def pifunc(n):
    T.<t> = PolynomialRing(QQ, 't')
    return T([0] + [
        sum(
            binomial(2*k, k-i) * i^(2*n) * (-1)^(i+1)
        for i in (1..k)) / (k * factorial(2*n-1))
    for k in (1..n)])

In [8]:
for n in (1..5):
    print(pifunc(n))

t
-t^2 + 1/6*t
t^3 - 1/4*t^2 + 1/120*t
-t^4 + 1/3*t^3 - 1/40*t^2 + 1/5040*t
t^5 - 5/12*t^4 + 7/144*t^3 - 17/12096*t^2 + 1/362880*t


Gives the coefficient of $p^J$ in $ph(\pi_t)$. The answer is a polynomial in $t$ of degree $\max(J)$. (Defined for any integer partition $J$.)

In [11]:
@cached_function
def picoeff(J):
    if len(J) == 0:
        return PolynomialRing(QQ, 't')(1)
    prefactor = (-1)^(sum(J) + len(J)) / product(factorial(m) for m in J.to_exp())
    return prefactor * sum(
        product(
            factorial(len(block) - 1) * pifunc(sum(block))
        for block in pi)
    for pi in set_partitions(J))

In [12]:
for n in (0..3):
    print({J: picoeff(J) for J in Partitions(n)})

{[]: 1}
{[1]: t}
{[2]: t^2 - 1/6*t, [1, 1]: 1/12*t}
{[3]: t^3 - 1/4*t^2 + 1/120*t, [2, 1]: 1/12*t^2 - 1/120*t, [1, 1, 1]: 1/360*t}


Gives $ph(\pi^r)$ in degress $0, \dots, n$, each as a polynomial in the Pontryagin classes.

In [16]:
@cached_function
def pi(r, n):
    Sym = SymmetricFunctions(QQ).elementary()
    return [Sym._from_dict({J: picoeff(J)[r] for J in Partitions(m)}) for m in (0..n)]

In [17]:
for r in (0..3):
    pretty_print(pi(r, 3))

[e[], 0, 0, 0]

[0,
 e[1],
 1/12*e[1, 1] - 1/6*e[2],
 1/360*e[1, 1, 1] - 1/120*e[2, 1] + 1/120*e[3]]

[0, 0, e[2], 1/12*e[2, 1] - 1/4*e[3]]

[0, 0, 0, e[3]]

### Fermionic anomaly polynomials

Gives $\hat{A}(R) \, ph(\pi^J)$ in degrees $0, \dots, n$, each as a polynomial in the Pontryagin classes. (Defined for any integer partition $J$.)

In [106]:
@cached_function
def fclass_total(J, n):
    Sym = SymmetricFunctions(QQ).elementary() # The Pontryagin classes.
    S.<s> = PolynomialRing(Sym, 's') # A formal degree-counting variable.
    Q = QuotientRing(S, s^(n+1)) # Truncate beyond degree n in s.

    # Compute Ahat * ph(pi^J), truncating beyond degree n.
    return (Q(ahat(n)) * prod([Q(pi(j, n)) for j in J])).lift().list()

In [107]:
fclass_total(Partition([]), 2)

[e[], -1/24*e[1], 7/5760*e[1, 1] - 1/1440*e[2]]

Gives $\hat{A}(R) \, ph(\pi^J)$ in degree $n$, as a polynomial in the Pontryagin classes. (Defined for any integer partition $J$.)

In [108]:
def fclass(J, n):
    """Returns the fermionic anomaly polynomial in 4n dimensions fclass = Ahat * ph(pi^J)."""
    return fclass_total(J, n)[n]

In [109]:
fclass(Partition([]), 1)

-1/24*e[1]

### Bosonic anomaly polynomials

In [87]:
@cached_function
def fclassmult(J, n):
    """Returns the smallest a >= 0 such that 2^a clears all even denominators in fclass(J, n)."""
    phi = fclass(J, n)
    denom = lcm(x.denominator() for x in phi.coefficients())
    return QQ.valuation(2)(denom)

In [85]:
fclassmult(Partition([]), 1)

3

In [86]:
@cached_function
def bclass(J, n):
    """Returns the bosonic anomaly polynomial in 4n dimensions bclass = fclass * 2^fclassmult."""
    return 2^fclassmult(J, n) * fclass(J, n)

In [79]:
bclass(Partition([]), 1)

-1/3*e[1]

### Structure of the map $p$

In [88]:
def partitions_leq(n):
    """Gives all partitions J with 0 <= n(J) <= n."""
    return (J for m in [0..n] for J in Partitions(m))

**Claim:** `fclassmult(J, n)` is given by $f(n - n(J))$ where $f(n)$ is the sequence https://oeis.org/A120738.

In [111]:
[[fclassmult(J, n) for J in partitions_leq(n)] for n in (1..5)]

[[3, 0],
 [7, 3, 0, 0],
 [10, 7, 3, 3, 0, 0, 0],
 [15, 10, 7, 7, 3, 3, 3, 0, 0, 0, 0, 0],
 [18, 15, 10, 10, 7, 7, 7, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0]]

In [123]:
[fclassmult(Partition([]), n) for n in (1..10)]

[3, 7, 10, 15, 18, 22, 25, 31, 34, 38]

Given an element of $\mathbb{Z}_{(2)}$, the ring of rationals with odd denominators, return its mod-2 reduction.

The result is always 0 or 1, because the fraction $p/q$ either has

* $p$ even, in which case $p/q$ is zero mod 2
* $p$ odd, in which case $1 - p/q$ is zero mod 2

We extend this operation to polynomials with coefficients in $\mathbb{Z}_{(2)}$. This includes all bosonic anomaly polynomials.

In [100]:
def reduce_mod2_2locally(x):
    if ZZ.is_parent_of(x):
        return x % 2
    elif QQ.is_parent_of(x):
        assert(is_odd(x.denominator()))
        return x.numerator() % 2
    elif SymmetricFunctions(QQ).elementary().is_parent_of(x):
        return x.map_coefficients(reduce_mod2_2locally)
    else:
        assert(False)

**Claim:** `bclass(J, n)` has mod-2 reduction $p^{J'}$, where $J'$ is $J$ padded with ones.

In [117]:
{J: reduce_mod2_2locally(bclass(J, 4)) for J in partitions_leq(4)}

{[]: e[1, 1, 1, 1],
 [1]: e[1, 1, 1, 1],
 [2]: e[2, 1, 1],
 [1, 1]: e[1, 1, 1, 1],
 [3]: e[3, 1],
 [2, 1]: e[2, 1, 1],
 [1, 1, 1]: e[1, 1, 1, 1],
 [4]: e[4],
 [3, 1]: e[3, 1],
 [2, 2]: e[2, 2],
 [2, 1, 1]: e[2, 1, 1],
 [1, 1, 1, 1]: e[1, 1, 1, 1]}

### Pontryagin numbers of spin manifolds

In [134]:
def fclassesmat(n):
    """Return the matrix of coefficients of all fclasses(J, n)."""
    return matrix([[fclass(J, n).coefficient(J2) for J2 in Partitions(n)] for J in partitions_leq(n)])

Each row is an anomaly polynomial $\hat{A}(R) \, ph(\pi^J)$ in $4n$ dimensions, its entries are the coefficients in the Pontryagin classes:

In [135]:
fclassesmat(3)

[  -1/60480  11/241920 -31/967680]
[     1/120     -1/480     1/1920]
[      -1/4       1/24          0]
[         0       -1/3        1/8]
[         1          0          0]
[         0          1          0]
[         0          0          1]

In [136]:
def remove1s(J):
    """Remove the 1s from a partition."""
    return Partition([j for j in J if j != 1])

In [137]:
def partitions_leq_no1s(n):
    """Gives all partitions J with 0 <= n(J) <= n with no 1s."""
    return (remove1s(J) for J in Partitions(n))

In [138]:
def fclassesmat2(n):
    """Same as fclasses, but restricted to those J with no 1s."""
    return matrix([[fclass(J, n).coefficient(J2) for J2 in Partitions(n)] for J in partitions_leq_no1s(n)])

In [139]:
fclassesmat2(3)

[         1          0          0]
[      -1/4       1/24          0]
[  -1/60480  11/241920 -31/967680]

In [140]:
def rational_smith_form(m):
    """Perform Smith decomposition on a rational matrix."""

    # Get lowest common denominator of elements.
    d = gcd(m.coefficients()).denominator()

    # Multiply by d to get an integer matrix.
    m = (d * m).change_ring(ZZ)

    # Run integer Smith decomposition.
    s, u, v = m.smith_form()

    # Divide by d again.
    s = s.change_ring(QQ) / d;

    return s, u, v

In [141]:
@cached_function
def spinpontryagins(n):
    """Get all Pontryagin numbers of spin manifolds of dimension 4n."""
    s, _, v = rational_smith_form(fclassesmat(n))
    return v * diagonal_matrix([x.denominator() for x in s.diagonal()]) * (1 if is_even(n) else 2)

Each column is a set of Pontryagin numbers of a spin manifold:

In [142]:
spinpontryagins(3)

[  3870720       192        -2]
[-13547520      -336       -12]
[-21288960      -576       -16]

This verifies that all their Dirac indices are integral, or even if $n$ is odd:

In [143]:
fclassesmat(3) * spinpontryagins(3)

[        2         0         0]
[    49392         2         0]
[ -1532160       -62         0]
[  1854720        40         2]
[  3870720       192        -2]
[-13547520      -336       -12]
[-21288960      -576       -16]

### Structure of the map $q$

In [146]:
def reduce_RP_r(r, l):
    Sym = SymmetricFunctions(QQ).elementary()
    return Sym._from_dict({Partition([a]): QQ(binomial(2 * l, r - a) * (-4)^(r - a)) for a in (0..r)})

In [147]:
def reduce_RP_J(J, l):
    Sym = SymmetricFunctions(QQ).elementary()
    if len(J) == 0:
        return Sym.one()
    
    return product(reduce_RP_r(j, l) for j in J)

In [148]:
def reduce_RP_theta(theta, l):
    Sym = SymmetricFunctions(QQ).elementary()
    if len(theta) == 0:
        return Sym.zero()
    
    assert(Sym.is_parent_of(theta))

    return sum(coeff * reduce_RP_J(J, l) for J, coeff in theta)

The following function evaluates $q \Big( \tfrac{1}{\gcd(2,n+1)} \Phi_{4n}(\slashed{D}_J) \Big)$ on all products $M_{4m} \times \mathbb{RP}^{4n - 4m - 2}$ where $M_{4m}$ is a spin manifold. This gives the subgroup $\mathbb{Z}_{2^v} \subseteq U(1)$ for some $v$; the function returns $v$. (Defined for $0 \leq m \leq n - 1$, and $J$ an integer partition with $n(J) \leq n$.)

In [154]:
def qpairing_on_products(J, n, m):
    """Return the subgroup of U(1) generated by q(Phi_4n(D_J) / gcd(n+1,2)) evaluated on all manifolds M_4m times RP^(4n - 4m - 2)."""
    assert(0 <= m < n)
    
    l = n - m
    red = reduce_RP_J(J, l)
    vec = vector([red.coefficient(J) for J in partitions_leq(m)])
    pairings = vec * fclassesmat(m) * spinpontryagins(m)
    etas = pairings / 2^(2 * l + (n % 2))
    val = max(QQ.valuation(2)(e.denominator()) for e in etas)

    return val

In [155]:
qpairing_on_products(Partition([]), 1, 0)

3

The following function evaluates $q \Big( \tfrac{1}{\gcd(2,n+1)} \Phi_{4n}(\slashed{D}_J) \Big)$ on all of $2\Omega^{\mathrm{Pin}^-}_{4n-2}(\mathrm{pt})$. (Return value and preconditions same as for previous function.)

In [156]:
def qpairing(J, n):
    """Return the subgroup of U(1) generated by q(2 Phi_4n(D_J) / gcd(n+1,2)) evaluated on all pin manifolds."""
    def dec_if_m_even(v, m):
        if is_odd(m):
            return v
        else:
            return max(v - 1, 0)
    
    return max(dec_if_m_even(qpairing_on_products(J, n, m), m) for m in (0..n-1))

**Claim:** `qpairing(J, n) + 1` is given by $2(n - n(J)) + (n \text{ mod } 2)$, matching the orders of the various generators of $\mathrm{pin}^-$ bordism:

In [168]:
[qpairing(Partition([]), n) for n in (1..6)]

[2, 3, 6, 7, 10, 11]

In [158]:
[qpairing(Partition([2]), n) for n in (1..8)]

[0, 0, 2, 3, 6, 7, 10, 11]

In [159]:
[qpairing(Partition([3]), n) for n in (1..8)]

[0, 0, 0, 1, 4, 5, 8, 9]

In [160]:
[qpairing(Partition([2, 2]), n) for n in (1..8)]

[0, 0, 0, 0, 2, 3, 6, 7]

In [161]:
[qpairing(Partition([3, 2]), n) for n in (1..8)]

[0, 0, 0, 0, 0, 1, 4, 5]

In [162]:
[qpairing(Partition([4]), n) for n in (1..8)]

[0, 0, 0, 0, 2, 3, 6, 7]